In [27]:
!pip install langchain-community langchain-text-splitters ollama langchain-ollama  pypdf  langchain-huggingface sentence-transformers faiss-cpu PyMuPDF   --no-deps

In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 51.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 53.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [28]:
import os
from google.colab import drive

# 1. 掛載 Google Drive
drive.mount('/content/drive')

# 2. 設定模型儲存路徑 (建議在 Drive 建立一個專門的資料夾)
# 這裡設定為 MyDrive 下的 RAG_project/ollama_models 資料夾
my_model_folder = '/content/drive/MyDrive/RAG/ollama_models'

# 如果資料夾不存在，建立它
if not os.path.exists(my_model_folder):
    os.makedirs(my_model_folder)

# 3. 【關鍵】設定環境變數，讓 Ollama 知道去哪裡找模型
os.environ['OLLAMA_MODELS'] = my_model_folder

print(f"Ollama 模型路徑已設定為: {os.environ['OLLAMA_MODELS']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ollama 模型路徑已設定為: /content/drive/MyDrive/RAG/ollama_models


In [29]:
import os
import subprocess
import time
from google.colab import drive


# --- 2. 修正並安裝 Ollama ---
print("正在安裝相依套件 zstd...")
!sudo apt-get install -y zstd  # <--- 新增這一行解決你的錯誤

print("正在安裝 Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 3. 啟動 Ollama 服務 ---
print("正在啟動 Ollama 服務...")
# 使用完整路徑以防萬一 (通常是 /usr/local/bin/ollama)
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 等待服務啟動
print("等待服務啟動中 (約 10 秒)...")
time.sleep(10)

# --- 4. 測試連線 ---
try:
    # 檢查服務是否活著
    check = subprocess.run(["curl", "-s", "http://localhost:11434"], capture_output=True, text=True)
    if "Ollama is running" in check.stdout:
        print("✅ 成功：Ollama 服務已在背景執行！")
    else:
        print("⚠️ 警告：服務似乎未回應，請檢查日誌。")
except Exception as e:
    print(f"檢查連線時發生錯誤: {e}")

正在安裝相依套件 zstd...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (496 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and direct

In [30]:
from langchain_ollama import ChatOllama
chat_model=ChatOllama(model="qwen3:4b-instruct-2507-q8_0",temperature=0)

In [9]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List,Tuple,Dict,Any
from langchain_community.document_loaders import PyPDFLoader
import fitz
from langchain_community.vectorstores import FAISS
import json
from pydantic import BaseModel,Field
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
chunk_size=1024
chunk_overlap=128
query="What the CRAG proposed to improve"

In [17]:

def replace_t_with_space(list_of_documents):


    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

def encode_pdf(path, chunk_size, chunk_overlap):


    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)
    for chunk in cleaned_texts:
        chunk.metadata['relevance_score'] = 1.0
    # Create embeddings and vector store
    vectorstore = FAISS.from_documents(cleaned_texts, embedding_model)

    return vectorstore
def encode_from_string(content, chunk_size, chunk_overlap):


    if not isinstance(content, str) or not content.strip():
        raise ValueError("Content must be a non-empty string.")

    try:
        # Split the content into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        chunks = text_splitter.create_documents([content])

        # Assign metadata to each chunk
        for chunk in chunks:
            chunk.metadata['relevance_score'] = 1.0

        # Generate embeddings and create the vector store

        vectorstore = FAISS.from_documents(chunks, embedding_model)

    except Exception as e:
        raise RuntimeError(f"An error occurred during the encoding process: {str(e)}")

    return vectorstore


In [18]:
vectorstore=encode_pdf(path,chunk_size,chunk_overlap)
retriever=vectorstore.as_retriever()

QA chain

In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

QA_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)

In [37]:
import os
def get_user_feedback(query, response, relevance, quality, comments=""):
    return {
        "query":query,
        "response":response,
        "relevance":relevance,
        "quality":int(quality),
        "comments":comments,
    }

def store_feedback(feedback):
  folder_path = "data"
  os.makedirs(folder_path, exist_ok=True)
  file_path = os.path.join(folder_path, "feedback_data.json")
  with open("data/feedback_data.json", "a") as f: #"a" = append 模式不會覆蓋原本資料
      json.dump(feedback,f) #將 Python 資料寫入檔案，並以 JSON 格式儲存
      f.write("\n")

def load_feedback_data():
    feedback_data=[]
    try:
        with open ("data/feedback_data.json", "r") as f:
            for line in f: # 一行一行讀
                feedback_data.append(json.loads(line.strip())) #json.loads(...)將 JSON 字串轉回 Python dict
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    return feedback_data

In [42]:
class Response(BaseModel):
    answer: str = Field(..., title="The answer to the question. The options can be only 'Yes' or 'No'")

def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]]) -> List[Any]:
    template="""
        Determine if the following feedback response is relevant to the current query and document content.
        You are also provided with the Feedback original query that was used to generate the feedback response.
        Current query: {query}
        Feedback query: {feedback_query}
        Document content: {doc_content}
        Feedback response: {feedback_response}

        Is this feedback relevant? Respond with only 'Yes' or 'No'.
    """
    relevance_prompt=PromptTemplate.from_template(
        template=template,
        #input_variables=["query","feedback_query","doc_content","feedback_response"],
    )

    relevance_chain=relevance_prompt | chat_model.with_structured_output(Response)

    for doc in docs:
        relevance_feedback=[]
        for feedback in feedback_data:
            input_data={
                "query":query,
                "feedback_query":feedback["query"],
                "doc_content":doc.page_content[:1000],
                "feedback_response":feedback["response"]
            }
            result=relevance_chain.invoke(input_data).answer
            if result is "yes":
                relevance_feedback.append(feedback)
        if relevance_feedback:
            avg_relevance = sum(f['relevance'] for f in relevance_feedback) / len(relevance_feedback)
            doc.metadata["relevance_score"] =doc.metadata["relevance_score"] *(avg_relevance / 3)

    return sorted(docs ,key=lambda x : x.metadata["relevance_score"],reverse=True)




<>:32: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:32: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
/tmp/ipython-input-3236151773.py:32: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if result is "yes":


In [39]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], texts: List[str]) -> Any:
    good_response=[f for f in feedback_data if f["relevance"] >= 4 and f["quality"] >= 4]

    additional_texts=[]
    for f in good_response:
        conbined_text=f["query"] +"  "+ f['response']
        additional_texts.append(conbined_text)

    additional_texts=" ".join(additional_texts)

    all_texts=texts + additional_texts
    new_vectorstore=encode_from_string(all_texts)
    return new_vectorstore

In [47]:
!ps aux | grep ollama

root        9563  0.0  0.2 1785788 33496 ?       Sl   13:26   0:00 ollama serve
root       14444  0.0  0.0   7376  3460 ?        S    13:45   0:00 /bin/bash -c ps aux | grep ollama
root       14446  0.0  0.0   6484  2304 ?        R    13:45   0:00 grep ollama


In [46]:
response=QA_chain.invoke(query)
relevance = 5
quality = 5
feedback = get_user_feedback(query, response, relevance, quality)
store_feedback(feedback)

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [49]:
docs = retriever.invoke(query)
adjusted_docs = adjust_relevance_scores(query, docs, load_feedback_data())

In [51]:
def print_docs(title, documents):
    print(f"\n{'='*20} {title} (共 {len(documents)} 筆) {'='*20}")

    for i, doc in enumerate(documents):
        # 1. 嘗試取得分數，如果沒有則顯示 N/A
        score = doc.metadata.get("relevance_score", "N/A")

        # 2. 為了版面整潔，內容只印出前 100 個字
        content_preview = doc.page_content[:300].replace('\n', ' ')

        print(f"📄 第 {i+1} 名")
        print(f"   🔹 分數 (Relevance Score): {score}")
        print(f"   🔹 來源 (Source): {doc.metadata.get('source', 'Unknown')}")
        print(f"   🔹 內容預覽: {content_preview}...")
        print("-" * 50)

# --- 執行列印 ---

# 1. 印出原始檢索結果 (通常順序是依照向量相似度，且可能還沒有 relevance_score)
print_docs("原始檢索結果 (Docs)", docs)

# 2. 印出調整後的結果 (應該會有分數，且順序可能改變)
print_docs("調整後結果 (Adjusted Docs)", adjusted_docs)


==================== 原始檢索結果 (Docs) (共 4 筆) ====================
📄 第 1 名
   🔹 分數 (Relevance Score): 1.0
   🔹 來源 (Source): /content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf
   🔹 內容預覽: knowledge retrieval actions discriminately. With the further leverage of web search and optimized knowledge utilization, CRAG has significantly improved the ability of automatic self-correction and efficient utilization of retrieved documents. Experiments extensively demonstrate its adaptabil- ity t...
--------------------------------------------------
📄 第 2 名
   🔹 分數 (Relevance Score): 1.0
   🔹 來源 (Source): /content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf
   🔹 內容預覽: Algorithm 1: CRAG Inference Require :E (Retrieval Evaluator), W (Query Rewriter), G (Generator) Input : x (Input question), D = {d1, d2, ..., dk} (Retrieved documents) Output : y (Generated response) 1 scorei = E evaluates the relevance of each pair (x, di), di ∈ D 2 Confidence = Calculate and give ...
--------------------------------------